In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ner-cleaned/kaggle_cleaned.csv
/kaggle/input/news-article/cleaned_file.csv


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

In [3]:
data = pd.read_csv("/kaggle/input/ner-cleaned/kaggle_cleaned.csv",encoding="latin")
data['unnamed_cleaned_articles'] = data['unnamed_cleaned_articles'].astype(str)

In [4]:
data_clean = data.copy()
sentiment_map = {'negative':0,'neutral':1,'positive':2}

data_clean['sentiments'] = data_clean['sentiments'].map(sentiment_map)

#data_clean['text'] = data_clean['article'].apply(lambda x: BeautifulSoup(x, "lxml").text)
data_clean['text'] = data_clean['unnamed_cleaned_articles'].apply(lambda x: BeautifulSoup(x, "lxml").text)

In [5]:
data_clean = data_clean.loc[:, ['text', 'sentiments']]

In [6]:
data_clean

,text,sentiments
0,"msft is l grow at a fast pace , and AI initiat...",2
1,own linkedin d . billion in from it pre subscr...,2
2,ha launch a copilot at for financ profession t...,1
3,'s path to becom the largest compani in the wo...,2
4,nondeal roadshow ndr allow institut investor t...,2
...,...,...
9264,ocbc sgx : O is one of three localist bank in ...,1
9265,"the earn season is come to a close , and it ha...",2
9266,ma ha announc that the dividenestrict on bank ...,1
9267,"sgx : ^sti , or sti , is a bellweth in that co...",1


In [7]:
neutral_mapping = {0:3,1:1,2:3}
data_clean['neutral_sentiments'] = data_clean['sentiments'].map(neutral_mapping)

In [8]:
data_clean_pn = data_clean[data_clean['sentiments'] != 0]

In [9]:
train_pn, test_pn = train_test_split(data_clean_pn, test_size=0.2, random_state=42)
X_train_pn = train_pn['text'].values
X_test_pn = test_pn['text'].values
y_train_pn = train_pn['sentiments']
y_test_pn = test_pn['sentiments']

In [10]:
train_neutral, test_neutral = train_test_split(data_clean, test_size=0.2, random_state=42)
X_train_neutral = train_neutral['text'].values
X_test_neutral = test_neutral['text'].values
y_train_neutral = train_neutral['neutral_sentiments']
y_test_neutral = test_neutral['neutral_sentiments']

In [11]:


def stem(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

en_stopwords = list(stopwords.words("english")) 

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(1, 1),
    stop_words = en_stopwords)

In [12]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [13]:
np.random.seed(42)

pipeline_svm_pn = make_pipeline(vectorizer, 
                            SVC(kernel='linear', gamma=0.5, probability=True))

grid_svm_pn = GridSearchCV(pipeline_svm_pn,
                    param_grid = {'svc__C': [ 0.1]}, 
                    cv = kfolds,
                    scoring="roc_auc",
                    verbose=1,   
                    n_jobs=-1) 

grid_svm_pn.fit(X_train_pn, y_train_pn)
grid_svm_pn.score(X_test_pn, y_test_pn)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extr

0.9080914942426335

In [16]:
np.random.seed(42)

pipeline_svm_neutral= make_pipeline(vectorizer, 
                            SVC(kernel='linear', gamma=0.5,probability=True))

grid_svm_neutral = GridSearchCV(pipeline_svm_neutral,
                    param_grid = {'svc__C': [ 0.1]}, 
                    cv = kfolds,
                    scoring="roc_auc",
                    verbose=1,   
                    n_jobs=-1) 

grid_svm_neutral.fit(X_train_neutral, y_train_neutral)
grid_svm_neutral.score(X_test_neutral, y_test_neutral)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extr

0.892468619811523

In [17]:

data_clean_bias = data_clean[data_clean['sentiments'] !=0]
random_rows = data_clean.sample(n=50, replace=False, random_state=42)  # random_state is optional, for reproducibility
random_rows_bias = data_clean_bias.sample(n=50, replace=False, random_state=42)

In [18]:
for index, row in random_rows_bias.iterrows():
    #print(row['text'])
    predictedPN = grid_svm_pn.predict_proba([row['text']])
    #print(predictedSM)
    if(predictedPN[0][1]>predictedPN[0][0]):
        new_value = 2
    else:
        new_value = 1
    random_rows_bias.at[index, 'predicted'] = new_value

In [19]:
random_rows_bias

,text,sentiments,neutral_sentiments,predicted
8600,fund trade index trade strategi trend video cl...,1,1,1.0
1984,"nyse : cvx , one of success of john D. rockefe...",1,1,2.0
2157,walmart confirm it will buy smaker vizio vzio ...,2,3,2.0
7132,"In the crypto world , the search for the best ...",2,3,2.0
3251,intuit ytd stock return Of . outperform by . a...,1,1,1.0
2023,entertain famili settl lawsuit against netflix...,1,1,2.0
4156,whi uber bn buyback is link to it boss 's big ...,2,3,2.0
1729,"abbv wa rtli in the news due to sever factor ,...",1,1,1.0
6619,", founder of cardano , recentlyd to comparison...",1,1,1.0
5339,announc today that itent into an agreement to ...,1,1,2.0


In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(random_rows_bias['sentiments'], random_rows_bias['predicted'])
print("Accuracy:", accuracy)

Accuracy: 0.78


In [21]:
for index, row in random_rows.iterrows():
    #print(row['text'])
    predicted = grid_svm_neutral.predict_proba([row['text']])
    #print(predictedSM)
    if(predicted[0][1]>predicted[0][0]):
        new_value = 3
    else:
        new_value = 1
    random_rows.at[index, 'predicted'] = new_value

In [23]:
accuracy = accuracy_score(random_rows['neutral_sentiments'], random_rows['predicted'])
print("Accuracy:", accuracy)

Accuracy: 0.8


In [ ]:
random_rows

In [ ]:
import pickle

filename_pos = '/kaggle/output/positive_model.sav'
filename_neg = '/kaggle/output/negative_model.sav'
pickle.dump(grid_svm_pn, open(filename_pos, 'wb'))
pickle.dump(grid_svm_nn, open(filename_neg, 'wb'))

In [ ]:
nn = [0.1,0.9]
pn = [0,8,0.2]